# Exploring Transactions Data

Source: [IBM Synthetic Credit Card Transactions Dataset on Kaggle](https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions)

In [1]:
import joblib
import pandas as pd
pd.set_option('display.max_columns', 100)

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tdf = joblib.load('data/transactions')
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24386900 entries, 0 to 24386899
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   User                  int64         
 1   Card                  int64         
 2   Time                  object        
 3   Amount                object        
 4   Use Chip              object        
 5   Merchant Name         int64         
 6   Merchant City         object        
 7   Merchant State        object        
 8   Zip                   float64       
 9   MCC                   int64         
 10  Errors?               object        
 11  Is Fraud?             object        
 12  transaction_datetime  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(7)
memory usage: 2.4+ GB


## Pre-Process

### Try to Save Space

In [3]:
tdf['MCC'] = tdf['MCC'].astype('int32')
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24386900 entries, 0 to 24386899
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   User                  int64         
 1   Card                  int64         
 2   Time                  object        
 3   Amount                object        
 4   Use Chip              object        
 5   Merchant Name         int64         
 6   Merchant City         object        
 7   Merchant State        object        
 8   Zip                   float64       
 9   MCC                   int32         
 10  Errors?               object        
 11  Is Fraud?             object        
 12  transaction_datetime  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(7)
memory usage: 2.3+ GB


In [6]:
tdf.drop(columns=['Merchant Name'], inplace=True)
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24386900 entries, 0 to 24386899
Data columns (total 12 columns):
 #   Column                Dtype         
---  ------                -----         
 0   User                  int64         
 1   Card                  int64         
 2   Time                  object        
 3   Amount                object        
 4   Use Chip              object        
 5   Merchant City         object        
 6   Merchant State        object        
 7   Zip                   float64       
 8   MCC                   int32         
 9   Errors?               object        
 10  Is Fraud?             object        
 11  transaction_datetime  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(7)
memory usage: 2.1+ GB


In [5]:
tdf.head(3)

,User,Card,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,transaction_datetime
0,0,0,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No,2002-09-01
1,0,0,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No,2002-09-01
2,0,0,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No,2002-09-02


In [7]:
tdf.loc[~tdf['Amount'].str.contains('$')].shape

(0, 12)

In [10]:
tdf['Amount'] = tdf['Amount'].apply(lambda value: float(value[1:]))
tdf['Amount'] = tdf['Amount'].astype('float32')
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24386900 entries, 0 to 24386899
Data columns (total 12 columns):
 #   Column                Dtype         
---  ------                -----         
 0   User                  int64         
 1   Card                  int64         
 2   Time                  object        
 3   Amount                float32       
 4   Use Chip              object        
 5   Merchant City         object        
 6   Merchant State        object        
 7   Zip                   float64       
 8   MCC                   int32         
 9   Errors?               object        
 10  Is Fraud?             object        
 11  transaction_datetime  datetime64[ns]
dtypes: datetime64[ns](1), float32(1), float64(1), int32(1), int64(2), object(6)
memory usage: 2.0+ GB


In [11]:
tdf['Use Chip'].value_counts()

Use Chip
Swipe Transaction     15386082
Chip Transaction       6287598
Online Transaction     2713220
Name: count, dtype: int64

In [12]:
tdf['Use Chip'] = tdf['Use Chip'].apply(lambda word: word.split()[0])
tdf['Use Chip'].value_counts()

Use Chip
Swipe     15386082
Chip       6287598
Online     2713220
Name: count, dtype: int64

### Combine Datetime Columns

In [5]:
tdf['Time'].sample(n=10)

13759342    15:12
14806318    10:40
6794274     06:57
8576284     08:58
18437265    09:37
10345942    08:33
22962013    14:15
6004777     22:02
8124608     08:02
12818088    16:47
Name: Time, dtype: object

In [6]:
tdf.describe()

,User,Card,Year,Month,Day,Merchant Name,Zip,MCC
count,2.438690e+07,2.438690e+07,2.438690e+07,2.438690e+07,2.438690e+07,2.438690e+07,2.150876e+07,2.438690e+07
mean,1.001019e+03,1.351366e+00,2.011955e+03,6.525064e+00,1.571812e+01,-4.769230e+17,5.095644e+04,5.561171e+03
std,5.694612e+02,1.407154e+00,5.105921e+00,3.472355e+00,8.794073e+00,4.758940e+18,2.939707e+04,8.793154e+02
min,0.000000e+00,0.000000e+00,1.991000e+03,1.000000e+00,1.000000e+00,-9.222899e+18,5.010000e+02,1.711000e+03
25%,5.100000e+02,0.000000e+00,2.008000e+03,3.000000e+00,8.000000e+00,-4.500543e+18,2.837400e+04,5.300000e+03
50%,1.006000e+03,1.000000e+00,2.013000e+03,7.000000e+00,1.600000e+01,-7.946765e+17,4.674200e+04,5.499000e+03
75%,1.477000e+03,2.000000e+00,2.016000e+03,1.000000e+01,2.300000e+01,3.189517e+18,7.756400e+04,5.812000e+03
max,1.999000e+03,8.000000e+00,2.020000e+03,1.200000e+01,3.100000e+01,9.223292e+18,9.992800e+04,9.402000e+03


In [3]:
from src.process_data import get_zero_padded

In [4]:
tdf['Month'] = tdf['Month'].apply(get_zero_padded)
tdf['Day'] = tdf['Day'].apply(get_zero_padded)
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24386900 entries, 0 to 24386899
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   User            int64  
 1   Card            int64  
 2   Year            int64  
 3   Month           object 
 4   Day             object 
 5   Time            object 
 6   Amount          object 
 7   Use Chip        object 
 8   Merchant Name   int64  
 9   Merchant City   object 
 10  Merchant State  object 
 11  Zip             float64
 12  MCC             int64  
 13  Errors?         object 
 14  Is Fraud?       object 
dtypes: float64(1), int64(5), object(9)
memory usage: 2.7+ GB


In [13]:
tdf['Month'].value_counts()

Month
01    2142220
12    2104435
10    2075188
08    2070407
07    2051785
11    2026832
05    2023412
09    2006307
03    2002867
06    1981230
02    1960411
04    1941806
Name: count, dtype: int64

In [7]:
tdf['transaction_datetime'] = pd.to_datetime(tdf[['Year', 'Month', 'Day']], format='%Y-%m-%d', errors='coerce')
tdf[['Year', 'Month', 'Day', 'transaction_datetime']].head(3)

,Year,Month,Day,transaction_datetime
0,2002,09,01,2002-09-01
1,2002,09,01,2002-09-01
2,2002,09,02,2002-09-02


In [8]:
tdf.drop(columns=['Year', 'Month', 'Day'], inplace=True)
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24386900 entries, 0 to 24386899
Data columns (total 13 columns):
 #   Column                Dtype         
---  ------                -----         
 0   User                  int64         
 1   Card                  int64         
 2   Time                  object        
 3   Amount                object        
 4   Use Chip              object        
 5   Merchant Name         int64         
 6   Merchant City         object        
 7   Merchant State        object        
 8   Zip                   float64       
 9   MCC                   int64         
 10  Errors?               object        
 11  Is Fraud?             object        
 12  transaction_datetime  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(7)
memory usage: 2.4+ GB


In [10]:
tdf[['Hour', 'Minute']] = tdf['Time'].str.split(':', expand=True)
tdf.head(2)

## Save

In [13]:
joblib.dump(tdf, 'data/transactions')

['data/transactions']